In [ ]:
import sys
from pathlib import Path
path_root = '../'
sys.path.append(str(path_root))

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from one.api import ONE
from brainbox.io.one import SessionLoader, SpikeSortingLoader
from iblatlas.atlas import AllenAtlas
from utils.ibl_data_utils import (
    load_spiking_data, load_trials_and_mask, merge_probes,
    load_trial_behaviors, load_anytime_behaviors,
    prepare_data, 
    select_brain_regions, list_brain_regions, 
    bin_spiking_data, save_data
)

In [ ]:
# Instantiate ONE
ONE.setup(base_url='https://openalyx.internationalbrainlab.org', 
          silent=True, 
          cache_dir='/mnt/3TB/yizi/Downloads/ONE/openalyx.internationalbrainlab.org'
)
one = ONE(password='international')
ba = AllenAtlas()

In [ ]:
freeze_file = './data/2023_12_bwm_release.csv'
bwm_df = pd.read_csv(freeze_file, index_col=0)

In [ ]:
# Load an example session
idx = 400
pid = bwm_df.pid[idx]
eid, probe = one.pid2eid(pid)

In [ ]:
params = {
    # setup for trial decoding:
    'align_time': 'firstMovement_times',
    'time_window': (0., 1.),
    'binsize': 0.02,
    'single_region': False # use all available regions
}

In [ ]:
trials, mask = load_trials_and_mask(one, eid, min_rt=0.08, max_rt=2., nan_exclude='default')

In [ ]:
neural_dict, behave_dict, metadata = prepare_data(one, eid, bwm_df, params)

In [ ]:
regions, beryl_reg = list_brain_regions(neural_dict, **params)
region_cluster_ids = select_brain_regions(neural_dict, beryl_reg, regions, **params)

In [ ]:
binned_spikes, clusters_used_in_bins = bin_spiking_data(region_cluster_ids, neural_dict, trials, **params)

In [ ]:
trial_idx = 0
plt.figure(figsize=(4,3))
plt.imshow(binned_spikes[trial_idx].T, aspect="auto", cmap="binary")
plt.title(f"spike count in trial {trial_idx}")
plt.xlabel("time bin")
plt.ylabel("unit")
plt.colorbar();

In [ ]:
binned_behaviors = load_trial_behaviors(one, eid, trials, allow_nans=True, **params)

In [ ]:
behave = "left-pupil-diameter"
plt.figure(figsize=(4,3))
plt.plot(binned_behaviors[behave][trial_idx])
plt.title(f"{behave} in trial {trial_idx}")
plt.xlabel("time bin")
plt.ylabel(behave);

In [ ]:
save_data(eid, binned_spikes, binned_behaviors)